In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Reshape

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_path = './xray_chest2.tfr'

buffer_size = 5000
count = 20
batch_size = 20

epochs = 10

In [3]:
def parse(serialized):

    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        }

    # Parse the serialized data so we get a dict with our data.
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)

    # Get the image as raw bytes.
    image_raw = parsed_example['image']

    # Decode the raw bytes so it becomes a tensor with type.
    image = tf.decode_raw(image_raw, tf.float64)
    
    # Get the label associated with the image.
    label = parsed_example['label']

    # The image and label are now correct TensorFlow types.
    return image, label

In [4]:
dataset = tf.data.TFRecordDataset(data_path)

In [5]:
dataset = dataset.map(parse).map(lambda x, y : (tf.reshape(x, (300,300,1)),y)).batch(20)
 
    


In [6]:
iter = dataset.make_initializable_iterator()
el = iter.get_next()


with tf.Session() as sess:
    sess.run(iter.initializer)
    
    data = sess.run(el)


Instructions for updating:
Colocations handled automatically by placer.


In [7]:
data[0].shape

(20, 300, 300, 1)

In [9]:
model = keras.Sequential()
#add model layers

model.add(Conv2D(16, kernel_size=3, activation='relu',input_shape=(300,300,1)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 16)      160       
_________________________________________________________________
dropout (Dropout)            (None, 298, 298, 16)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1420864)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               181870720 
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
__________

In [18]:
model.fit(dataset.make_one_shot_iterator(), steps_per_epoch=30, epochs=epochs)

Epoch 1/10
30/30 [==============================] - 47s 2s/step - loss: 0.3799 - acc: 0.8400
Epoch 2/10
30/30 [==============================] - 46s 2s/step - loss: 0.3261 - acc: 0.8817
Epoch 3/10
30/30 [==============================] - 46s 2s/step - loss: 0.2304 - acc: 0.9300
Epoch 4/10
30/30 [==============================] - 46s 2s/step - loss: 0.2353 - acc: 0.9150
Epoch 5/10
30/30 [==============================] - 45s 2s/step - loss: 0.2256 - acc: 0.9117
Epoch 6/10
30/30 [==============================] - 45s 1s/step - loss: 0.2138 - acc: 0.9267
Epoch 7/10
30/30 [==============================] - 45s 2s/step - loss: 0.1898 - acc: 0.9417
Epoch 8/10
30/30 [==============================] - 46s 2s/step - loss: 0.1599 - acc: 0.9433
Epoch 9/10
 0/30 [..............................] - ETA: 0s - loss: 0.0000e+00 - acc: 0.9255

In [19]:
data[1][:10]

array([0, 1, 1, 0, 1, 1, 0, 1, 0, 1])

In [23]:
import numpy as np

In [24]:
np.round(model.predict(data[0][0:10]))

array([[0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [15]:
data[1].sum()/len(data[1])

0.7

In [26]:
model.save('pneumounia.h5')

In [25]:
keras.losses